## 📈 **1. PERFORMANCE DE VENDAS**

### Perguntas Respondidas:
1. Qual é o volume total de vendas por mês e trimestre?
2. Quais são os produtos com melhor performance de vendas?
3. Como o desconto impacta no volume de vendas?

In [0]:


%sql
-- PERGUNTA 1.1: Volume total de vendas por mês e trimestre
SELECT 
    d.year,
    d.quarter,
    d.month_name,
    d.month,
    COUNT(f.order_key) as total_orders,
    SUM(f.quantity) as total_items_sold,
    COUNT(DISTINCT f.customer_key) as unique_customers,
    ROUND(SUM(f.total_amount), 2) as total_revenue,
    ROUND(AVG(f.total_amount), 2) as avg_order_value,
    ROUND(SUM(f.gross_amount), 2) as gross_revenue,
    ROUND(SUM(f.discount_amount), 2) as total_discounts_given,
    ROUND(SUM(f.total_amount) - LAG(SUM(f.total_amount)) OVER (ORDER BY d.year, d.month), 2) as revenue_mom_change,
    ROUND((SUM(f.total_amount) - LAG(SUM(f.total_amount)) OVER (ORDER BY d.year, d.month)) / 
          LAG(SUM(f.total_amount)) OVER (ORDER BY d.year, d.month) * 100, 2) as revenue_mom_growth_pct
FROM workspace.amazon_sales_mvp.fact_sales f
JOIN workspace.amazon_sales_mvp.dim_date d ON f.date_key = d.date_key
WHERE f.order_status NOT IN ('CANCELLED', 'RETURNED')
GROUP BY d.year, d.quarter, d.month_name, d.month
ORDER BY d.year, d.month;

In [0]:
%sql
--PERGUNTA 1.2: Produtos com melhor performance de vendas
SELECT 
    p.product_name,
    p.category,
    p.brand,
    p.price_category,
    -- Métricas de Volume
    COUNT(f.order_key) as times_ordered,
    SUM(f.quantity) as total_quantity_sold,
    COUNT(DISTINCT f.customer_key) as unique_customers_bought,
    -- Métricas Financeiras
    ROUND(AVG(f.unit_price), 2) as avg_selling_price,
    ROUND(SUM(f.total_amount), 2) as total_revenue,
    ROUND(AVG(f.total_amount), 2) as avg_order_value,
    ROUND(SUM(f.discount_amount), 2) as total_discounts_applied,
    -- Performance Relativa
    ROUND(SUM(f.total_amount) / SUM(SUM(f.total_amount)) OVER() * 100, 2) as revenue_share_pct,
    ROUND(SUM(f.quantity) / SUM(SUM(f.quantity)) OVER() * 100, 2) as volume_share_pct,
    -- Rentabilidade
    ROUND((SUM(f.total_amount) - SUM(f.discount_amount)) / COUNT(f.order_key), 2) as avg_net_revenue_per_order
FROM workspace.amazon_sales_mvp.fact_sales f
JOIN workspace.amazon_sales_mvp.dim_product p ON f.product_key = p.product_id
WHERE f.order_status NOT IN ('CANCELLED', 'RETURNED')
GROUP BY p.product_id, p.product_name, p.category, p.brand, p.price_category
ORDER BY total_revenue DESC
LIMIT 20;

In [0]:
%sql
--PERGUNTA 1.3: Como o desconto impacta no volume de vendas
SELECT 
    CASE 
        WHEN f.discount_rate = 0 THEN '0% - No Discount'
        WHEN f.discount_rate <= 0.05 THEN '1-5% Discount'
        WHEN f.discount_rate <= 0.10 THEN '6-10% Discount'
        WHEN f.discount_rate <= 0.15 THEN '11-15% Discount'
        WHEN f.discount_rate <= 0.25 THEN '16-25% Discount'
        ELSE '25%+ High Discount'
    END as discount_bracket,
    -- Métricas de Volume
    COUNT(f.order_key) as total_orders,
    ROUND(COUNT(f.order_key) * 100.0 / SUM(COUNT(f.order_key)) OVER(), 2) as pct_of_total_orders,
    SUM(f.quantity) as total_items_sold,
    -- Métricas de Desconto
    ROUND(AVG(f.discount_rate * 100), 2) as avg_discount_pct,
    ROUND(MIN(f.discount_rate * 100), 2) as min_discount_pct,
    ROUND(MAX(f.discount_rate * 100), 2) as max_discount_pct,
    -- Impacto Financeiro
    ROUND(SUM(f.gross_amount), 2) as gross_revenue,
    ROUND(SUM(f.discount_amount), 2) as total_discount_given,
    ROUND(SUM(f.total_amount), 2) as net_revenue,
    ROUND(AVG(f.total_amount), 2) as avg_order_value,
    -- Análise de Efetividade
    ROUND(SUM(f.discount_amount) / SUM(f.gross_amount) * 100, 2) as discount_impact_pct,
    ROUND(SUM(f.total_amount) / COUNT(f.order_key), 2) as revenue_per_order,
    -- Customer Behavior
    COUNT(DISTINCT f.customer_key) as unique_customers,
    ROUND(COUNT(f.order_key) / COUNT(DISTINCT f.customer_key), 2) as avg_orders_per_customer
FROM workspace.amazon_sales_mvp.fact_sales f
WHERE f.order_status NOT IN ('CANCELLED', 'RETURNED')
GROUP BY 
    CASE 
        WHEN f.discount_rate = 0 THEN '0% - No Discount'
        WHEN f.discount_rate <= 0.05 THEN '1-5% Discount'
        WHEN f.discount_rate <= 0.10 THEN '6-10% Discount'
        WHEN f.discount_rate <= 0.15 THEN '11-15% Discount'
        WHEN f.discount_rate <= 0.25 THEN '16-25% Discount'
        ELSE '25%+ High Discount'
    END
ORDER BY avg_discount_pct;

---

## 👥 **2. COMPORTAMENTO DO CLIENTE**

### Perguntas Respondidas:
1. Quais países/regiões geram mais receita?
2. Qual é o ticket médio por cliente?
3. Quais são os métodos de pagamento mais utilizados?

In [0]:
%sql
--PERGUNTA 2.1: Países/regiões que geram mais receita
SELECT 
    c.country,
    c.state,
    -- Métricas de Base de Clientes
    COUNT(DISTINCT c.customer_id) as unique_customers,
    COUNT(f.order_key) as total_orders,
    SUM(f.quantity) as total_items_purchased,
    -- Métricas Financeiras
    ROUND(SUM(f.total_amount), 2) as total_revenue,
    ROUND(AVG(f.total_amount), 2) as avg_order_value,
    ROUND(SUM(f.total_amount) / COUNT(DISTINCT c.customer_id), 2) as revenue_per_customer,
    -- Comportamento do Cliente
    ROUND(COUNT(f.order_key) / COUNT(DISTINCT c.customer_id), 2) as avg_orders_per_customer,
    ROUND(SUM(f.quantity) / COUNT(DISTINCT c.customer_id), 2) as avg_items_per_customer,
    -- Performance Relativa
    ROUND(SUM(f.total_amount) / SUM(SUM(f.total_amount)) OVER() * 100, 2) as revenue_share_pct,
    ROUND(COUNT(DISTINCT c.customer_id) / SUM(COUNT(DISTINCT c.customer_id)) OVER() * 100, 2) as customer_share_pct,
    -- Insights de Mercado
    CASE 
        WHEN SUM(f.total_amount) / COUNT(DISTINCT c.customer_id) > 500 THEN 'High Value Market'
        WHEN SUM(f.total_amount) / COUNT(DISTINCT c.customer_id) > 200 THEN 'Medium Value Market'
        ELSE 'Volume Market'
    END as market_category
FROM workspace.amazon_sales_mvp.dim_customer c
JOIN workspace.amazon_sales_mvp.fact_sales f ON c.customer_id = f.customer_key
WHERE f.order_status NOT IN ('CANCELLED', 'RETURNED')
GROUP BY c.country, c.state
ORDER BY total_revenue DESC;

In [0]:
%sql
-- PERGUNTA 2.2: Ticket médio por cliente
SELECT 
    c.customer_name,
    c.country,
    c.state,
    c.customer_segment,
    -- Métricas de Atividade
    COUNT(f.order_key) as total_orders,
    SUM(f.quantity) as total_items_bought,
    -- Métricas Financeiras
    ROUND(SUM(f.total_amount), 2) as total_spent,
    ROUND(AVG(f.total_amount), 2) as avg_order_value,
    ROUND(SUM(f.total_amount) / COUNT(f.order_key), 2) as ticket_medio,
    ROUND(MIN(f.total_amount), 2) as min_order_value,
    ROUND(MAX(f.total_amount), 2) as max_order_value,
    -- Análise Temporal
    MIN(d.order_date) as first_purchase,
    MAX(d.order_date) as last_purchase,
    DATEDIFF(MAX(d.order_date), MIN(d.order_date)) as customer_lifetime_days,
    -- Customer Value Analysis
    ROUND(try_divide(SUM(f.total_amount), DATEDIFF(MAX(d.order_date), MIN(d.order_date))), 2) as avg_daily_spend,
    ROUND(SUM(f.discount_amount), 2) as total_discounts_received,
    ROUND(try_divide(SUM(f.discount_amount), SUM(f.total_amount)) * 100, 2) as discount_rate_pct,
    -- Classificação de Cliente
    CASE 
        WHEN SUM(f.total_amount) > 2000 THEN 'VIP Customer'
        WHEN SUM(f.total_amount) > 1000 THEN 'High Value'
        WHEN SUM(f.total_amount) > 500 THEN 'Medium Value'
        ELSE 'Standard Customer'
    END as customer_value_tier
FROM workspace.amazon_sales_mvp.dim_customer c
JOIN workspace.amazon_sales_mvp.fact_sales f ON c.customer_id = f.customer_key
JOIN workspace.amazon_sales_mvp.dim_date d ON f.date_key = d.date_key
WHERE f.order_status NOT IN ('CANCELLED', 'RETURNED')
GROUP BY c.customer_id, c.customer_name, c.country, c.state, c.customer_segment
ORDER BY total_spent DESC
LIMIT 50;

In [0]:
%sql
-- PERGUNTA 2.3: Métodos de pagamento mais utilizados
SELECT 
    f.payment_method,
    -- Métricas de Uso
    COUNT(f.order_key) as total_transactions,
    ROUND(COUNT(f.order_key) * 100.0 / SUM(COUNT(f.order_key)) OVER(), 2) as percentage_of_transactions,
    SUM(f.quantity) as total_items,
    -- Métricas Financeiras
    ROUND(SUM(f.total_amount), 2) as total_revenue,
    ROUND(AVG(f.total_amount), 2) as avg_transaction_value,
    ROUND(SUM(f.total_amount) * 100.0 / SUM(SUM(f.total_amount)) OVER(), 2) as percentage_of_revenue,
    -- Análise por Valor de Transação
    ROUND(MIN(f.total_amount), 2) as min_transaction_value,
    ROUND(MAX(f.total_amount), 2) as max_transaction_value,
    ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY f.total_amount), 2) as median_transaction_value,
    -- Comportamento por Método
    COUNT(DISTINCT f.customer_key) as unique_customers,
    ROUND(COUNT(f.order_key) / COUNT(DISTINCT f.customer_key), 2) as avg_transactions_per_customer,
    -- Insights Geográficos (baseado em padrões conhecidos)
    CASE f.payment_method
        WHEN 'UPI' THEN 'India-Focused'
        WHEN 'Credit Card' THEN 'Global/Developed Markets'
        WHEN 'Debit Card' THEN 'Global/General Use'
        WHEN 'Cash on Delivery' THEN 'Emerging Markets'
        ELSE 'Digital/Modern'
    END as likely_market_focus,
    -- Classificação por Volume
    CASE 
        WHEN COUNT(f.order_key) > (SELECT COUNT(*) * 0.25 FROM workspace.amazon_sales_mvp.fact_sales WHERE order_status NOT IN ('CANCELLED', 'RETURNED')) THEN 'Primary Method'
        WHEN COUNT(f.order_key) > (SELECT COUNT(*) * 0.10 FROM workspace.amazon_sales_mvp.fact_sales WHERE order_status NOT IN ('CANCELLED', 'RETURNED')) THEN 'Secondary Method'
        ELSE 'Niche Method'
    END as usage_category
FROM workspace.amazon_sales_mvp.fact_sales f
WHERE f.order_status NOT IN ('CANCELLED', 'RETURNED')
GROUP BY f.payment_method
ORDER BY total_transactions DESC;

---

## 📦 **3. ANÁLISE DE PRODUTOS**

### Perguntas Respondidas:
1. Quais categorias de produtos são mais vendidas?
2. Qual é a distribuição de preços por categoria?
3. Quais marcas têm melhor performance?

In [0]:
%sql
-- PERGUNTA 3.1: Categorias de produtos mais vendidas
SELECT 
    p.category,
    -- Métricas de Portfolio
    COUNT(DISTINCT p.product_id) as unique_products,
    COUNT(f.order_key) as total_orders,
    SUM(f.quantity) as total_units_sold,
    -- Métricas Financeiras
    ROUND(SUM(f.total_amount), 2) as total_revenue,
    ROUND(AVG(f.total_amount), 2) as avg_order_value,
    ROUND(AVG(f.unit_price), 2) as avg_product_price,
    ROUND(SUM(f.total_amount) / SUM(f.quantity), 2) as revenue_per_unit,
    -- Performance Relativa
    ROUND(SUM(f.total_amount) / SUM(SUM(f.total_amount)) OVER() * 100, 2) as revenue_share_pct,
    ROUND(SUM(f.quantity) / SUM(SUM(f.quantity)) OVER() * 100, 2) as volume_share_pct,
    ROUND(COUNT(f.order_key) / SUM(COUNT(f.order_key)) OVER() * 100, 2) as order_share_pct,
    -- Customer Engagement
    COUNT(DISTINCT f.customer_key) as unique_customers,
    ROUND(COUNT(f.order_key) / COUNT(DISTINCT f.customer_key), 2) as avg_orders_per_customer,
    ROUND(SUM(f.quantity) / COUNT(DISTINCT f.customer_key), 2) as avg_units_per_customer,
    -- Análise de Desconto
    ROUND(AVG(f.discount_rate * 100), 2) as avg_discount_pct,
    ROUND(SUM(f.discount_amount), 2) as total_discounts_given,
    -- Classificação de Performance
    CASE 
        WHEN SUM(f.total_amount) > (SELECT SUM(total_amount) * 0.20 FROM workspace.amazon_sales_mvp.fact_sales WHERE order_status NOT IN ('CANCELLED', 'RETURNED')) THEN 'Star Category'
        WHEN SUM(f.total_amount) > (SELECT SUM(total_amount) * 0.10 FROM workspace.amazon_sales_mvp.fact_sales WHERE order_status NOT IN ('CANCELLED', 'RETURNED')) THEN 'Strong Performer'
        WHEN SUM(f.total_amount) > (SELECT SUM(total_amount) * 0.05 FROM workspace.amazon_sales_mvp.fact_sales WHERE order_status NOT IN ('CANCELLED', 'RETURNED')) THEN 'Solid Contributor'
        ELSE 'Niche Category'
    END as category_performance_tier
FROM workspace.amazon_sales_mvp.fact_sales f
JOIN workspace.amazon_sales_mvp.dim_product p ON f.product_key = p.product_id
WHERE f.order_status NOT IN ('CANCELLED', 'RETURNED')
GROUP BY p.category
ORDER BY total_revenue DESC;

In [0]:
%sql
-- PERGUNTA 3.2: Distribuição de preços por categoria
SELECT 
    p.category,
    -- Métricas Básicas de Preço
    COUNT(DISTINCT p.product_id) as products_in_category,
    ROUND(MIN(f.unit_price), 2) as min_price,
    ROUND(MAX(f.unit_price), 2) as max_price,
    ROUND(AVG(f.unit_price), 2) as avg_price,
    ROUND(STDDEV(f.unit_price), 2) as price_std_deviation,
    -- Análise de Percentis
    ROUND(PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY f.unit_price), 2) as price_25th_percentile,
    ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY f.unit_price), 2) as median_price,
    ROUND(PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY f.unit_price), 2) as price_75th_percentile,
    ROUND(PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY f.unit_price), 2) as price_90th_percentile,
    -- Segmentação por Preço
    SUM(CASE WHEN f.unit_price <= 50 THEN 1 ELSE 0 END) as products_under_50,
    SUM(CASE WHEN f.unit_price BETWEEN 50 AND 200 THEN 1 ELSE 0 END) as products_50_200,
    SUM(CASE WHEN f.unit_price BETWEEN 200 AND 500 THEN 1 ELSE 0 END) as products_200_500,
    SUM(CASE WHEN f.unit_price > 500 THEN 1 ELSE 0 END) as products_over_500,
    -- Volume vs Preço Analysis
    ROUND(SUM(CASE WHEN f.unit_price <= 50 THEN f.quantity ELSE 0 END) / SUM(f.quantity) * 100, 2) as volume_pct_under_50,
    ROUND(SUM(CASE WHEN f.unit_price > 500 THEN f.quantity ELSE 0 END) / SUM(f.quantity) * 100, 2) as volume_pct_over_500,
    -- Revenue vs Price Analysis
    ROUND(SUM(CASE WHEN f.unit_price <= 50 THEN f.total_amount ELSE 0 END) / SUM(f.total_amount) * 100, 2) as revenue_pct_under_50,
    ROUND(SUM(CASE WHEN f.unit_price > 500 THEN f.total_amount ELSE 0 END) / SUM(f.total_amount) * 100, 2) as revenue_pct_over_500,
    -- Classificação da Categoria
    CASE 
        WHEN AVG(f.unit_price) > 400 THEN 'Premium Category'
        WHEN AVG(f.unit_price) > 200 THEN 'Mid-Range Category'
        WHEN AVG(f.unit_price) > 100 THEN 'Standard Category'
        ELSE 'Budget Category'
    END as price_positioning
FROM workspace.amazon_sales_mvp.fact_sales f
JOIN workspace.amazon_sales_mvp.dim_product p ON f.product_key = p.product_id
WHERE f.order_status NOT IN ('CANCELLED', 'RETURNED')
GROUP BY p.category
ORDER BY avg_price DESC;

In [0]:
%sql
-- PERGUNTA 3.3: Marcas com melhor performance
SELECT 
    p.brand,
    -- Portfolio da Marca
    COUNT(DISTINCT p.product_id) as unique_products,
    COUNT(DISTINCT p.category) as categories_present,
    -- Performance de Vendas
    COUNT(f.order_key) as total_orders,
    SUM(f.quantity) as total_units_sold,
    ROUND(SUM(f.total_amount), 2) as total_revenue,
    ROUND(AVG(f.total_amount), 2) as avg_order_value,
    ROUND(AVG(f.unit_price), 2) as avg_product_price,
    -- Customer Engagement
    COUNT(DISTINCT f.customer_key) as unique_customers,
    ROUND(COUNT(f.order_key) / COUNT(DISTINCT f.customer_key), 2) as avg_orders_per_customer,
    ROUND(SUM(f.total_amount) / COUNT(DISTINCT f.customer_key), 2) as avg_revenue_per_customer,
    -- Market Share
    ROUND(SUM(f.total_amount) / SUM(SUM(f.total_amount)) OVER() * 100, 2) as revenue_market_share_pct,
    ROUND(SUM(f.quantity) / SUM(SUM(f.quantity)) OVER() * 100, 2) as volume_market_share_pct,
    -- Pricing Strategy
    ROUND(MIN(f.unit_price), 2) as lowest_price_product,
    ROUND(MAX(f.unit_price), 2) as highest_price_product,
    ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY f.unit_price), 2) as median_product_price,
    -- Discount Analysis
    ROUND(AVG(f.discount_rate * 100), 2) as avg_discount_pct,
    ROUND(SUM(f.discount_amount), 2) as total_discounts_given,
    -- Performance Classification
    CASE 
        WHEN SUM(f.total_amount) > (SELECT SUM(total_amount) * 0.15 FROM workspace.amazon_sales_mvp.fact_sales WHERE order_status NOT IN ('CANCELLED', 'RETURNED')) THEN 'Market Leader'
        WHEN SUM(f.total_amount) > (SELECT SUM(total_amount) * 0.05 FROM workspace.amazon_sales_mvp.fact_sales WHERE order_status NOT IN ('CANCELLED', 'RETURNED')) THEN 'Strong Brand'
        WHEN SUM(f.total_amount) > (SELECT SUM(total_amount) * 0.02 FROM workspace.amazon_sales_mvp.fact_sales WHERE order_status NOT IN ('CANCELLED', 'RETURNED')) THEN 'Established Brand'
        ELSE 'Niche Brand'
    END as brand_tier,
    -- Strategy Classification
    CASE 
        WHEN AVG(f.unit_price) > 300 THEN 'Premium Brand'
        WHEN AVG(f.unit_price) > 150 THEN 'Mid-Market Brand'
        ELSE 'Value Brand'
    END as pricing_strategy
FROM workspace.amazon_sales_mvp.fact_sales f
JOIN workspace.amazon_sales_mvp.dim_product p ON f.product_key = p.product_id
WHERE f.order_status NOT IN ('CANCELLED', 'RETURNED')
GROUP BY p.brand
ORDER BY total_revenue DESC;

---

## ⚙️ **4. ANÁLISE OPERACIONAL**

### Perguntas Respondidas:
1. Qual é a taxa de cancelamento e devolução?
2. Como os custos de frete impactam na margem?
3. Qual é a distribuição de status dos pedidos?

In [0]:
%sql
-- PERGUNTA 4.1: Taxa de cancelamento e devolução
-- Análise Detalhada por Status
SELECT 
    f.order_status,
    -- Métricas Básicas
    COUNT(f.order_key) as total_orders,
    ROUND(COUNT(f.order_key) * 100.0 / SUM(COUNT(f.order_key)) OVER(), 2) as percentage_of_total,
    SUM(f.quantity) as total_items,
    -- Impacto Financeiro
    ROUND(SUM(f.total_amount), 2) as total_amount_impact,
    ROUND(AVG(f.total_amount), 2) as avg_order_value,
    ROUND(SUM(f.total_amount) * 100.0 / SUM(SUM(f.total_amount)) OVER(), 2) as revenue_impact_pct,
    -- Customer Analysis
    COUNT(DISTINCT f.customer_key) as unique_customers_affected,
    ROUND(COUNT(f.order_key) / COUNT(DISTINCT f.customer_key), 2) as avg_orders_per_customer,
    -- Product Analysis
    COUNT(DISTINCT f.product_key) as unique_products_affected,
    -- Operational Insights
    CASE f.order_status
        WHEN 'DELIVERED' THEN 'Success - Revenue Generated'
        WHEN 'SHIPPED' THEN 'In Progress - Revenue Pending'
        WHEN 'CANCELLED' THEN 'Lost Revenue - Process Issue'
        WHEN 'RETURNED' THEN 'Quality Issue - Customer Dissatisfaction'
        ELSE 'Other Status'
    END as business_impact_type
FROM workspace.amazon_sales_mvp.fact_sales f
GROUP BY f.order_status
ORDER BY total_orders DESC;

-- ============================================================================
-- Análise Consolidada: Sucesso vs Falha
-- ============================================================================
SELECT 
    CASE 
        WHEN f.order_status IN ('DELIVERED', 'SHIPPED') THEN 'SUCCESSFUL_ORDERS'
        WHEN f.order_status IN ('CANCELLED', 'RETURNED') THEN 'FAILED_ORDERS'
        ELSE 'OTHER_STATUS'
    END as order_outcome,
    COUNT(f.order_key) as total_orders,
    ROUND(COUNT(f.order_key) * 100.0 / SUM(COUNT(f.order_key)) OVER(), 2) as percentage,
    ROUND(SUM(f.total_amount), 2) as total_revenue_impact,
    ROUND(AVG(f.total_amount), 2) as avg_order_value,
    -- KPIs Operacionais
    COUNT(DISTINCT f.customer_key) as customers_affected,
    COUNT(DISTINCT f.product_key) as products_affected,
    -- Análise por Método de Pagamento
    COUNT(DISTINCT f.payment_method) as payment_methods_used
FROM workspace.amazon_sales_mvp.fact_sales f
GROUP BY 
    CASE 
        WHEN f.order_status IN ('DELIVERED', 'SHIPPED') THEN 'SUCCESSFUL_ORDERS'
        WHEN f.order_status IN ('CANCELLED', 'RETURNED') THEN 'FAILED_ORDERS'
        ELSE 'OTHER_STATUS'
    END
ORDER BY total_orders DESC;

In [0]:
%sql
--PERGUNTA 4.2: Como os custos de frete impactam na margem
SELECT 
    CASE 
        WHEN f.shipping_cost = 0 THEN 'Free Shipping'
        WHEN f.shipping_cost <= 5 THEN 'Low Shipping ($0-$5)'
        WHEN f.shipping_cost <= 10 THEN 'Medium Shipping ($5-$10)'
        WHEN f.shipping_cost <= 15 THEN 'High Shipping ($10-$15)'
        ELSE 'Premium Shipping ($15+)'
    END as shipping_category,
    -- Volume Metrics
    COUNT(f.order_key) as total_orders,
    ROUND(COUNT(f.order_key) * 100.0 / SUM(COUNT(f.order_key)) OVER(), 2) as pct_of_total_orders,
    SUM(f.quantity) as total_items_shipped,
    -- Shipping Cost Analysis
    ROUND(AVG(f.shipping_cost), 2) as avg_shipping_cost,
    ROUND(MIN(f.shipping_cost), 2) as min_shipping_cost,
    ROUND(MAX(f.shipping_cost), 2) as max_shipping_cost,
    ROUND(SUM(f.shipping_cost), 2) as total_shipping_revenue,
    -- Order Value Analysis
    ROUND(SUM(f.total_amount), 2) as total_order_value,
    ROUND(AVG(f.total_amount), 2) as avg_order_value,
    ROUND(SUM(f.net_amount), 2) as net_revenue_after_shipping,
    -- Margin Impact Analysis
    ROUND(AVG(f.shipping_cost) / AVG(f.total_amount) * 100, 2) as shipping_as_pct_of_order,
    ROUND(SUM(f.shipping_cost) / SUM(f.total_amount) * 100, 2) as shipping_revenue_pct,
    -- Customer Behavior
    COUNT(DISTINCT f.customer_key) as unique_customers,
    ROUND(AVG(f.total_amount - f.shipping_cost), 2) as avg_product_value_excl_shipping,
    -- Profitability Insights
    CASE 
        WHEN AVG(f.shipping_cost) = 0 THEN 'Free Shipping Strategy - Customer Acquisition Focus'
        WHEN AVG(f.shipping_cost) / AVG(f.total_amount) < 0.05 THEN 'Low Shipping Impact - Good Margins'
        WHEN AVG(f.shipping_cost) / AVG(f.total_amount) < 0.15 THEN 'Moderate Shipping Impact'
        ELSE 'High Shipping Impact - Review Strategy'
    END as margin_impact_assessment,
    -- Business Intelligence
    ROUND(AVG(CASE WHEN f.shipping_cost = 0 THEN f.total_amount ELSE NULL END), 2) as avg_order_with_free_shipping,
    ROUND(COUNT(CASE WHEN f.total_amount > 100 THEN 1 END) * 100.0 / COUNT(*), 2) as pct_orders_over_100
FROM workspace.amazon_sales_mvp.fact_sales f
WHERE f.order_status NOT IN ('CANCELLED', 'RETURNED')
GROUP BY 
    CASE 
        WHEN f.shipping_cost = 0 THEN 'Free Shipping'
        WHEN f.shipping_cost <= 5 THEN 'Low Shipping ($0-$5)'
        WHEN f.shipping_cost <= 10 THEN 'Medium Shipping ($5-$10)'
        WHEN f.shipping_cost <= 15 THEN 'High Shipping ($10-$15)'
        ELSE 'Premium Shipping ($15+)'
    END
ORDER BY avg_order_value DESC;

In [0]:
%sql
-- PERGUNTA 4.3: Distribuição de status dos pedidos
-- Análise Cruzada: Status vs Payment Method
SELECT 
    f.order_status,
    f.payment_method,
    -- Volume Metrics
    COUNT(f.order_key) as order_count,
    ROUND(COUNT(f.order_key) * 100.0 / SUM(COUNT(f.order_key)) OVER(PARTITION BY f.order_status), 2) as pct_within_status,
    ROUND(COUNT(f.order_key) * 100.0 / SUM(COUNT(f.order_key)) OVER(PARTITION BY f.payment_method), 2) as pct_within_payment_method,
    -- Financial Impact
    ROUND(SUM(f.total_amount), 2) as total_value,
    ROUND(AVG(f.total_amount), 2) as avg_order_value,
    -- Operational Insights
    COUNT(DISTINCT f.customer_key) as unique_customers,
    COUNT(DISTINCT f.product_key) as unique_products,
    -- Risk Assessment
    CASE 
        WHEN f.order_status IN ('CANCELLED', 'RETURNED') AND COUNT(f.order_key) > 5 THEN 'High Risk Combination'
        WHEN f.order_status IN ('CANCELLED', 'RETURNED') THEN 'Monitor Closely'
        WHEN f.order_status IN ('DELIVERED', 'SHIPPED') THEN 'Successful Combination'
        ELSE 'Standard Process'
    END as risk_assessment
FROM workspace.amazon_sales_mvp.fact_sales f
GROUP BY f.order_status, f.payment_method
ORDER BY f.order_status, order_count DESC;

-- ============================================================================
-- Status Distribution with Geographic Insights
-- ============================================================================
SELECT 
    f.order_status,
    c.country,
    -- Performance Metrics
    COUNT(f.order_key) as orders,
    ROUND(COUNT(f.order_key) * 100.0 / SUM(COUNT(f.order_key)) OVER(PARTITION BY c.country), 2) as pct_within_country,
    ROUND(AVG(f.total_amount), 2) as avg_order_value,
    -- Customer Impact
    COUNT(DISTINCT f.customer_key) as customers_affected,
    -- Operational Efficiency by Region
    CASE 
        WHEN f.order_status = 'DELIVERED' THEN 'Successful Delivery'
        WHEN f.order_status = 'SHIPPED' THEN 'In Transit'
        WHEN f.order_status = 'CANCELLED' THEN 'Operational Issue'
        WHEN f.order_status = 'RETURNED' THEN 'Quality/Satisfaction Issue'
        ELSE 'Other'
    END as operational_category
FROM workspace.amazon_sales_mvp.fact_sales f
JOIN workspace.amazon_sales_mvp.dim_customer c ON f.customer_key = c.customer_id
GROUP BY f.order_status, c.country
HAVING COUNT(f.order_key) > 1  -- Filter for meaningful data
ORDER BY c.country, orders DESC;

---

## ⏰ **5. ANÁLISE TEMPORAL**

### Perguntas Respondidas:
1. Existe sazonalidade nas vendas?
2. Quais são os períodos de maior volume de pedidos?

In [0]:
%sql
-- PERGUNTA 5.1: Existe sazonalidade nas vendas?
-- Análise Mensal Detalhada
SELECT 
    d.month,
    d.month_name,
    d.quarter,
    -- Performance Metrics
    COUNT(f.order_key) as total_orders,
    SUM(f.quantity) as total_items,
    ROUND(SUM(f.total_amount), 2) as total_revenue,
    ROUND(AVG(f.total_amount), 2) as avg_order_value,
    -- Sazonalidade Analysis
    ROUND(SUM(f.total_amount) - AVG(SUM(f.total_amount)) OVER(), 2) as revenue_vs_monthly_avg,
    ROUND((SUM(f.total_amount) / AVG(SUM(f.total_amount)) OVER() - 1) * 100, 2) as pct_above_below_avg,
    -- Customer Behavior
    COUNT(DISTINCT f.customer_key) as unique_customers,
    ROUND(COUNT(f.order_key) / COUNT(DISTINCT f.customer_key), 2) as avg_orders_per_customer,
    -- Product Mix Analysis
    COUNT(DISTINCT f.product_key) as unique_products_sold,
    ROUND(AVG(f.discount_rate * 100), 2) as avg_discount_pct,
    -- Seasonal Classification
    CASE 
        WHEN d.month IN (11, 12) THEN 'Holiday Season'
        WHEN d.month IN (6, 7, 8) THEN 'Summer Season'
        WHEN d.month IN (1, 2) THEN 'Post-Holiday'
        WHEN d.month IN (3, 4, 5) THEN 'Spring Season'
        ELSE 'Fall Season'
    END as seasonal_period,
    -- Performance Ranking
    RANK() OVER (ORDER BY SUM(f.total_amount) DESC) as revenue_rank,
    RANK() OVER (ORDER BY COUNT(f.order_key) DESC) as volume_rank
FROM workspace.amazon_sales_mvp.fact_sales f
JOIN workspace.amazon_sales_mvp.dim_date d ON f.date_key = d.date_key
WHERE f.order_status NOT IN ('CANCELLED', 'RETURNED')
GROUP BY d.month, d.month_name, d.quarter
ORDER BY d.month;

-- ============================================================================
-- Análise Trimestral com Tendências
-- ============================================================================
SELECT 
    d.year,
    d.quarter,
    -- Core Metrics
    COUNT(f.order_key) as total_orders,
    COUNT(DISTINCT f.customer_key) as unique_customers,
    COUNT(DISTINCT f.product_key) as unique_products,
    SUM(f.quantity) as total_items_sold,
    ROUND(SUM(f.total_amount), 2) as total_revenue,
    ROUND(AVG(f.total_amount), 2) as avg_order_value,
    ROUND(SUM(f.total_amount) / COUNT(DISTINCT f.customer_key), 2) as revenue_per_customer,
    -- Year-over-Year Analysis
    LAG(SUM(f.total_amount)) OVER (PARTITION BY d.quarter ORDER BY d.year) as prev_year_same_quarter_revenue,
    ROUND((SUM(f.total_amount) - LAG(SUM(f.total_amount)) OVER (PARTITION BY d.quarter ORDER BY d.year)) / 
          LAG(SUM(f.total_amount)) OVER (PARTITION BY d.quarter ORDER BY d.year) * 100, 2) as yoy_growth_pct,
    -- Quarter-over-Quarter Analysis  
    LAG(SUM(f.total_amount)) OVER (ORDER BY d.year, d.quarter) as prev_quarter_revenue,
    ROUND((SUM(f.total_amount) - LAG(SUM(f.total_amount)) OVER (ORDER BY d.year, d.quarter)) / 
          LAG(SUM(f.total_amount)) OVER (ORDER BY d.year, d.quarter) * 100, 2) as qoq_growth_pct,
    -- Seasonal Insights
    CASE d.quarter
        WHEN 1 THEN 'Q1 - New Year Start'
        WHEN 2 THEN 'Q2 - Spring/Summer Prep'
        WHEN 3 THEN 'Q3 - Summer Peak'
        WHEN 4 THEN 'Q4 - Holiday Season'
    END as quarter_characteristics
FROM workspace.amazon_sales_mvp.fact_sales f
JOIN workspace.amazon_sales_mvp.dim_date d ON f.date_key = d.date_key
WHERE f.order_status NOT IN ('CANCELLED', 'RETURNED')
GROUP BY d.year, d.quarter
ORDER BY d.year, d.quarter;

In [0]:
%sql 
-- PERGUNTA 5.2: Quais são os períodos de maior volume de pedidos?
-- Análise por Dia da Semana
SELECT 
    d.day_of_week,
    d.day_name,
    d.is_weekend,
    -- Volume Metrics
    COUNT(f.order_key) as total_orders,
    ROUND(COUNT(f.order_key) * 100.0 / SUM(COUNT(f.order_key)) OVER(), 2) as pct_of_weekly_orders,
    ROUND(AVG(COUNT(f.order_key)) OVER(), 2) as avg_daily_orders,
    SUM(f.quantity) as total_items,
    -- Revenue Metrics
    ROUND(SUM(f.total_amount), 2) as total_revenue,
    ROUND(AVG(f.total_amount), 2) as avg_order_value,
    ROUND(SUM(f.total_amount) * 100.0 / SUM(SUM(f.total_amount)) OVER(), 2) as pct_of_weekly_revenue,
    -- Customer Behavior
    COUNT(DISTINCT f.customer_key) as unique_customers,
    ROUND(COUNT(f.order_key) / COUNT(DISTINCT f.customer_key), 2) as avg_orders_per_customer,
    -- Day Classification
    CASE 
        WHEN d.is_weekend = 1 THEN 'Weekend Shopping'
        WHEN d.day_of_week = 2 THEN 'Monday - Week Start'
        WHEN d.day_of_week IN (3, 4, 5) THEN 'Midweek - Work Days'
        WHEN d.day_of_week = 6 THEN 'Friday - Week End'
        ELSE 'Other'
    END as day_category,
    -- Performance vs Average
    ROUND((COUNT(f.order_key) - AVG(COUNT(f.order_key)) OVER()) / AVG(COUNT(f.order_key)) OVER() * 100, 2) as orders_vs_avg_pct,
    -- Shopping Pattern Insights
    CASE 
        WHEN COUNT(f.order_key) > AVG(COUNT(f.order_key)) OVER() * 1.2 THEN 'High Volume Day'
        WHEN COUNT(f.order_key) < AVG(COUNT(f.order_key)) OVER() * 0.8 THEN 'Low Volume Day'
        ELSE 'Average Volume Day'
    END as volume_classification
FROM workspace.amazon_sales_mvp.fact_sales f
JOIN workspace.amazon_sales_mvp.dim_date d ON f.date_key = d.date_key
WHERE f.order_status NOT IN ('CANCELLED', 'RETURNED')
GROUP BY d.day_of_week, d.day_name, d.is_weekend
ORDER BY d.day_of_week;

-- ============================================================================
-- Análise de Picos Temporais (Anos e Crescimento)
-- ============================================================================
SELECT 
    d.year,
    -- Annual Performance
    COUNT(f.order_key) as total_orders,
    COUNT(DISTINCT f.customer_key) as unique_customers,
    COUNT(DISTINCT f.product_key) as unique_products_sold,
    ROUND(SUM(f.total_amount), 2) as total_revenue,
    ROUND(AVG(f.total_amount), 2) as avg_order_value,
    -- Growth Analysis
    LAG(COUNT(f.order_key)) OVER (ORDER BY d.year) as prev_year_orders,
    LAG(SUM(f.total_amount)) OVER (ORDER BY d.year) as prev_year_revenue,
    -- Year-over-Year Growth
    ROUND((COUNT(f.order_key) - LAG(COUNT(f.order_key)) OVER (ORDER BY d.year)) / 
          LAG(COUNT(f.order_key)) OVER (ORDER BY d.year) * 100, 2) as orders_yoy_growth_pct,
    ROUND((SUM(f.total_amount) - LAG(SUM(f.total_amount)) OVER (ORDER BY d.year)) / 
          LAG(SUM(f.total_amount)) OVER (ORDER BY d.year) * 100, 2) as revenue_yoy_growth_pct,
    -- Customer Acquisition
    ROUND((COUNT(DISTINCT f.customer_key) - LAG(COUNT(DISTINCT f.customer_key)) OVER (ORDER BY d.year)) / 
          LAG(COUNT(DISTINCT f.customer_key)) OVER (ORDER BY d.year) * 100, 2) as customer_growth_pct,
    -- Business Maturity Insights
    CASE 
        WHEN d.year = (SELECT MIN(year) FROM workspace.amazon_sales_mvp.dim_date d2 JOIN workspace.amazon_sales_mvp.fact_sales f2 ON d2.date_key = f2.date_key 
                       WHERE f2.order_status NOT IN ('CANCELLED', 'RETURNED')) 
        THEN 'Launch Year'
        WHEN ROW_NUMBER() OVER (ORDER BY d.year) <= 2 THEN 'Early Growth'
        ELSE 'Established Period'
    END as business_phase,
    -- Market Expansion
    ROUND(COUNT(DISTINCT f.product_key) / LAG(COUNT(DISTINCT f.product_key)) OVER (ORDER BY d.year) - 1, 2) as product_expansion_rate
FROM workspace.amazon_sales_mvp.fact_sales f
JOIN workspace.amazon_sales_mvp.dim_date d ON f.date_key = d.date_key
WHERE f.order_status NOT IN ('CANCELLED', 'RETURNED')
GROUP BY d.year
ORDER BY d.year;

---

## 📊 **Resumo Final**

### ✅ **Total de Consultas Implementadas: 16**

| Área de Análise | Consultas | Foco Principal |
|---|---|---|
| **📈 Performance de Vendas** | 3 | Volume, produtos top, impacto desconto |
| **👥 Comportamento Cliente** | 3 | Regional, ticket médio, pagamentos |
| **📦 Análise de Produtos** | 3 | Categorias, preços, marcas |
| **⚙️ Análise Operacional** | 3 | Cancelamentos, frete, status |
| **⏰ Análise Temporal** | 4 | Sazonalidade, picos, tendências |

### 🎯 **Características Técnicas:**

✅ **Arquitetura Medalhão** - Uso completo da Gold Layer  
✅ **Star Schema** - Joins otimizados com fact_sales e dimensões  
✅ **Métricas Avançadas** - Cálculos de YoY, MoM, market share  
✅ **Business Intelligence** - Classificações automáticas e insights  
✅ **Data Quality** - Filtros apropriados e validações  
✅ **Performance** - Queries otimizadas para grandes volumes  

### 🚀 **Como Utilizar:**

1. **Databricks Environment:** Execute cada célula SQL no ambiente Databricks
2. **Análise Sequencial:** Siga a ordem das seções para análise completa
3. **Customização:** Ajuste filtros e agregações conforme necessário
4. **Visualização:** Use resultados para criar dashboards no Power BI/Tableau
5. **Monitoramento:** Implemente como queries agendadas para KPIs

### 💡 **Insights Esperados:**

- **Sazonalidade:** Identificação de picos e vales de vendas
- **Performance:** Produtos e categorias campeãs
- **Comportamento:** Padrões de compra por região e segmento
- **Eficiência:** Métricas operacionais e oportunidades de otimização
- **Crescimento:** Tendências históricas e projeções

---

